In [4]:
import pandas as pd
import vaex
import glob
import gc
import metadata

In [5]:
for file in glob.glob('data/Estabelecimentos*.zip'):
    filename = file.split('/')[-1].split('.')[0]
    print(f'{filename}')
    df_cnpj = pd.read_csv(f'{file}', 
                    compression='zip', 
                    sep=';', 
                    encoding_errors='replace', 
                    header=None, 
                    names=list(metadata.CAMPOS.keys()),
                    dtype=metadata.CAMPOS,
                    parse_dates=['data_situacao_cadastral'])
    print(f'{df_cnpj.shape}')
    df_cnpj = vaex.from_pandas(df_cnpj)
    df_cnpj.export_hdf5(f'data/{filename}.hdf5')
    del df_cnpj
    print(f'{filename} exportado!')
    gc.collect()
    # break

Estabelecimentos7
(4753435, 30)
Estabelecimentos7 exportado!
Estabelecimentos8
(4753435, 30)
Estabelecimentos8 exportado!
Estabelecimentos9
(4753435, 30)
Estabelecimentos9 exportado!
Estabelecimentos2
(4753435, 30)
Estabelecimentos2 exportado!
Estabelecimentos4
(4753435, 30)
Estabelecimentos4 exportado!
Estabelecimentos3
(4753435, 30)
Estabelecimentos3 exportado!
Estabelecimentos5
(4753435, 30)
Estabelecimentos5 exportado!
Estabelecimentos1
(4753435, 30)
Estabelecimentos1 exportado!
Estabelecimentos6
(4753435, 30)
Estabelecimentos6 exportado!
Estabelecimentos0
(13032276, 30)
Estabelecimentos0 exportado!


In [6]:
df_estabelecimentos = vaex.concat([vaex.open(df) for df in glob.glob('data/Estabelecimentos*.hdf5')])

In [8]:
df_estabelecimentos.export_hdf5('data/brasil_cnpj.hdf5')

In [7]:
df_estabelecimentos.shape

(55813191, 30)

In [9]:
!rm -Rf data/Estabelecimentos?.zip

In [10]:
df_estabelecimentos.data_situacao_cadastral.str.len().value_counts()


8    55666179
1      147012
dtype: int64

## Separar 2000 estabelecimentos do município de São Paulo 


In [2]:
df_municipios = pd.read_csv('data/Municipios.zip',
                            compression='zip', 
                            sep=';', 
                            encoding_errors='replace',
                            names=['cd_municipio', 'nome_municipio'])

In [3]:
df_municipios

,cd_municipio,nome_municipio
0,1,GUAJARA-MIRIM
1,2,ALTO ALEGRE DOS PARECIS
2,3,PORTO VELHO
3,4,BURITIS
4,5,JI-PARANA
...,...,...
5566,9989,SERRA ALTA
5567,9991,TUNAPOLIS
5568,9993,GUARINOS
5569,9995,RIO QUENTE


In [40]:
df_municipios[df_municipios.nome_municipio == 'SAO PAULO']

,cd_municipio,nome_municipio
4140,7107,SAO PAULO


In [46]:
df_2000_sp = df_estabelecimentos[df_estabelecimentos.logradouro_cod_municipio == '7107'].sample(n=2000, random_state=13)

In [48]:
df_2000_sp.export_csv('results/2000_estabelecimentos_aleatorios_municipio_sao_paulo.csv')